<center> <h1> Restoration of underwater images </h1> <br> <h3> <br> 4MEO Computer vision 2</h3> </center>

# I. Introduction

### 1. Import libraries

In [1]:
import numpy as np
import scipy as sc
import cv2 as cv
import matplotlib.image as mpimg
import matplotlib.pylab as plt

# II. Compute inputs

## 1. Show function

In [2]:
def show(final):
    cv.imshow('White balanced', final)
    cv.waitKey(0)
    cv.destroyAllWindows()
    for i in range (1,5):
        cv.waitKey(1)

## 2. White balance function definition 

In [3]:
def white_balance(img):
    result = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv.cvtColor(result, cv.COLOR_LAB2BGR)
    return result

## 3. Enhance contrast function definition 

In [4]:
def enhance_contrast(im):
    rgb_img = im

    # convert from RGB color-space to YCrCb
    ycrcb_img = cv.cvtColor(rgb_img, cv.COLOR_BGR2YCrCb)

    # equalize the histogram of the Y channel
    ycrcb_img[:, :, 0] = cv.equalizeHist(ycrcb_img[:, :, 0])

    # convert back to RGB color-space from YCrCb
    equalized_img = cv.cvtColor(ycrcb_img, cv.COLOR_YCrCb2BGR)
    return equalized_img

## 4. Luminance weight map

In [ ]:
def lumWeightMap(im):
    L=np.mean(im,3) #mean for each axes
    lumWeightMap = np.sqrt((1/3)*(im[:,:,0]-L)^2+(im[:,:,1]-L)^2+(im[:,:,2]-L)^2)
    return lumWeightMap

## Output

In [5]:
img = cv.imread('data/test4.png')
firstInput = white_balance(img)
secondInput = enhance_contrast(firstInput)
final = np.hstack((img, firstInput, secondInput))
show(final)